# Jahn-Teller distortion with pressure in KCuF$_3$

In this example, we are studying variable-pressure diffraction data in KCuF$_3$, previously published in Zhou, et al. "Jahn–Teller distortion in perovskite KCuF$_3$ under high pressure", Journal of Fluorine Chemistry, 2011

## Import packages 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymatgen
import sys
import os
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
sys.path.insert(0,r"..\code")
from van_vleck_calculator import *

In [ ]:
%matplotlib notebook

## Import structures from Rietveld refinement

In [ ]:
filenames = [i for i in os.listdir() if i[-3:]=='cif']

In [ ]:
from pymatgen.core.structure import Structure

strucs = [Structure.from_file(i) for i in filenames]

In [ ]:
pressure = [i/1000.0 for i in [0.101325, 800, 1300, 2500, 3500, 4500, 5500, 6500, 8000]] # units are GPa

## Prepare data

In [ ]:
octahedral_volume = []

Q2_kanamori = []
Q3_kanamori = []

Q2 = []
Q3 = []
Q4 = []
Q5 = []
Q6 = []

phi = []
rho = []

BLDI = []
ECoN = []
QuadElon = []
BAV = []

pJT_dist = []
pJT_metric = []

delta_shear = []
delta_antishear = []
shear_fraction = []

In [ ]:
for i in range(len(strucs)):
    
    oct1 = Octahedron(strucs[i][5],strucs[i],possible_ligands=["F"])
    
    octahedral_volume.append(oct1.volume)
    
    van_vleck_modes_no_angular_distortion = oct1.calculate_van_vleck_distortion_modes(
        output_pairs = False,
        ignore_angular_distortion = True
    )
    Q2_kanamori.append(van_vleck_modes_no_angular_distortion[1])
    Q3_kanamori.append(van_vleck_modes_no_angular_distortion[2])
    
    van_vleck_modes = oct1.calculate_van_vleck_distortion_modes(output_pairs = False)
    Q2.append(van_vleck_modes[1])
    Q3.append(van_vleck_modes[2])
    Q4.append(van_vleck_modes[3])
    Q5.append(van_vleck_modes[4])
    Q6.append(van_vleck_modes[5])
    
    JT_params = oct1.calculate_van_vleck_jahn_teller_params(output_pairs = False)
    phi.append(JT_params[1])
    rho.append(JT_params[0])
    
    BLDI.append(
        oct1.calculate_bond_length_distortion_index()
    )
    ECoN.append(
        oct1.calculate_effective_coordination_number()
    )
    QuadElon.append(
        oct1.calculate_quadratic_elongation()
    )
    BAV.append(
        oct1.calculate_bond_angle_variance()
    )
    
    pJT_dist.append(
        oct1.calculate_off_centering_distance()
    )
    pJT_metric.append(
        oct1.calculate_off_centering_metric()
    )

    delta_shear.append(
        oct1.calculate_angular_shear_magnitude()
    )
    
    delta_antishear.append(
        oct1.calculate_angular_antishear_magnitude()
    )
    
    shear_fraction.append(
        oct1.calculate_shear_fraction_angular_distortion()
    )

In [ ]:
df = pd.DataFrame(
    {
        "P (GPa)":pressure,
        "Octahedral volume (Å^3)":octahedral_volume,
        "BLDI":BLDI,
        "ECoN":ECoN,
        "QuadElon":QuadElon,
        "Bond angle variance (°^2)":BAV,
#         "Q_2 (Å) [ignore angular distortion]":Q2_kanamori,
        "Q_2 (Å)":Q2,
#         "Q_3 (Å) [ignore angular distortion]":Q3_kanamori,
        "Q_3 (Å)":Q3,
        "Q_4 (Å)":Q4,
        "Q_5 (Å)":Q5,
        "Q_6 (Å)":Q6,
        "ρ (Å)":rho,
        "φ (°)":phi,
#         "off-centering distance (Å)":pJT_dist,
#         "off-centering metric (Å)":pJT_metric,
        "angular shear (°)":delta_shear,
        "angular antishear (°)":delta_antishear,
        "η":shear_fraction,
    }
)

## Check rotation is successful

Note: here, the rotation can be judged to have worked if the points are as close as possible to the x-, y-, and z- axes.

In [ ]:
Octahedron(strucs[0][5],strucs[0]).visualise_sites_for_van_vleck()

## print data

In [ ]:
df

In [ ]:
print(df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.3f}".format))